In [3]:
# IMPORTACIONES

import pandas as pd 
import json
import re

In [4]:
# FUNCIONES IMPORTANTES

# Para verificar tipos de datos y porcentajes
def verificar_tipo_dato(dataframe):
    diccionario = {'nombre_campo': [], 'tipo_dato': [], '%_no_nulos': [], '%_nulos': [], 'nulos': []}
    for columna in dataframe.columns:
        porcentaje_no_nulos = (dataframe[columna].count()/len(dataframe))*100
        diccionario['nombre_campo'].append(columna)
        diccionario['tipo_dato'].append(dataframe[columna].apply(type).unique())
        diccionario['%_no_nulos'].append(round(porcentaje_no_nulos,2))
        diccionario['%_nulos'].append(round(100-porcentaje_no_nulos,2))
        diccionario['nulos'].append(dataframe[columna].isnull().sum())
    
    df_info = pd.DataFrame(diccionario)
    return df_info





# Para ver la existencia de duplicados
def verificar_duplicados_columna(df, columna):
    lista_vacia = []
    filas_duplicadas = df[df.duplicated(subset=columna, keep=False)]
    for elemento in df[columna]:
        if len(filas_duplicadas) == len(lista_vacia):
            return 'No hay elementos duplicados'
        else:
            return filas_duplicadas.sort_values(by=columna, ascending=True)




# Para obtener año de lanzamiento
def obtener_año_lanzamiento(fecha):
    if pd.notna(fecha):
        if re.match(r'^\d{4}-\d{2}-\d{2}$', fecha):
            return fecha.split('-')[0]
    return 'Dato no disponible'




# Para reemplazar a float
def reemplazar_a_float(value):
    if pd.isna(value):
        return 0.0
    try:
        float_value = float(value)
        return float_value
    except:
        return 0.0

In [5]:
# EXTRACCION DE DATOS

# Se extraen los datos de los JSON y se les convierte en dataframe
archivo_games = 'C:\\Users\\cquir\\OneDrive\\Escritorio\\Data Science SH\\Proyecto Individual 1\\bases de datos\\output_steam_games.json'

# Se lee el dataset por linea
filas_games = []
with open(archivo_games) as f:
    for line in f.readlines():
        data = json.loads(line)
        filas_games.append(data)
dfgames = pd.DataFrame(filas_games)
dfgames

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,False,773640,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,False,733530,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
